In [ ]:
import pandas as pd
import numpy as np

# Read the CSV files
df = pd.read_csv('concept_region_Store_type_data2.csv')
df_store_region = pd.read_csv('concept_region_store_type_data3.csv')

# Define regions
regions = ['central', 'western', 'eastern', 'ecomm']

# ============================================================================
# STEP 1: Transform data into pivot format with region columns
# ============================================================================
df_pivot = df[['CONCEPT', 'STORE_TYPE', 'REGION', 
               'YEAR_TO_DATE_SALES_CURRENT_YEAR', 
               'GROWTH_PERCENTAGE', 
               'ACHIEVEMENT_PERCENTAGE']].copy()

concept_store_combos = df_pivot[['CONCEPT', 'STORE_TYPE']].drop_duplicates().sort_values(['CONCEPT', 'STORE_TYPE'])

result_data_with_totals = []

for _, row in concept_store_combos.iterrows():
    concept = row['CONCEPT']
    store_type = row['STORE_TYPE']
    
    result_row = {'CONCEPT': concept, 'STORE_TYPE': store_type}
    
    # For each region, get the metrics
    for region in regions:
        region_data = df_pivot[(df_pivot['CONCEPT'] == concept) & 
                               (df_pivot['STORE_TYPE'] == store_type) & 
                               (df_pivot['REGION'] == region)]
        
        if not region_data.empty:
            result_row[f'{region}_ytd_sales_cy'] = region_data['YEAR_TO_DATE_SALES_CURRENT_YEAR'].values[0]
            result_row[f'{region}_growth_pct'] = region_data['GROWTH_PERCENTAGE'].values[0]
            result_row[f'{region}_achievement_pct'] = region_data['ACHIEVEMENT_PERCENTAGE'].values[0]
        else:
            result_row[f'{region}_ytd_sales_cy'] = None
            result_row[f'{region}_growth_pct'] = None
            result_row[f'{region}_achievement_pct'] = None
    
    # Add Concept+Store Type totals
    concept_store_data = df[(df['CONCEPT'] == concept) & (df['STORE_TYPE'] == store_type)].iloc[0]
    result_row['combined_ytd_sales'] = concept_store_data['KSA_YTD_SALES_CONCEPT_STORE']
    result_row['combined_growth_pct'] = concept_store_data['KSA_GROWTH_PCT_CONCEPT_STORE']
    result_row['combined_achievement_pct'] = concept_store_data['KSA_ACHIEVEMENT_CONCEPT_STORE']
    
    result_data_with_totals.append(result_row)

result_df_with_totals = pd.DataFrame(result_data_with_totals)

# ============================================================================
# STEP 2: Add total rows (Concept totals after each concept)
# ============================================================================
final_rows = []
concepts_list = result_df_with_totals['CONCEPT'].unique()

for concept in concepts_list:
    # Add all rows for this concept
    concept_rows = result_df_with_totals[result_df_with_totals['CONCEPT'] == concept]
    final_rows.extend(concept_rows.to_dict('records'))
    
    # Add Concept+Region total row
    concept_region_row = {'CONCEPT': concept, 'STORE_TYPE': 'Total'}
    
    for region in regions:
        concept_region_data = df[(df['CONCEPT'] == concept) & (df['REGION'] == region)]
        if not concept_region_data.empty:
            first_row = concept_region_data.iloc[0]
            concept_region_row[f'{region}_ytd_sales_cy'] = first_row['KSA_YTD_SALES_CONCEPT_REGION']
            concept_region_row[f'{region}_growth_pct'] = first_row['KSA_GROWTH_PCT_CONCEPT_REGION']
            concept_region_row[f'{region}_achievement_pct'] = first_row['KSA_ACHIEVEMENT_CONCEPT_REGION']
        else:
            concept_region_row[f'{region}_ytd_sales_cy'] = None
            concept_region_row[f'{region}_growth_pct'] = None
            concept_region_row[f'{region}_achievement_pct'] = None
    
    # Concept totals
    concept_data = df[df['CONCEPT'] == concept].iloc[0]
    concept_region_row['combined_ytd_sales'] = concept_data['KSA_YTD_SALES_CONCEPT']
    concept_region_row['combined_growth_pct'] = concept_data['KSA_GROWTH_PCT_CONCEPT']
    concept_region_row['combined_achievement_pct'] = concept_data['KSA_ACHIEVEMENT_CONCEPT']
    
    final_rows.append(concept_region_row)

# ============================================================================
# STEP 3: Add KSA section (Store Type totals)
# ============================================================================
store_types = ['ecomm', 'mall', 'standalone']

for store_type in store_types:
    store_region_row = {'CONCEPT': 'KSA', 'STORE_TYPE': store_type.upper()}
    
    for region in regions:
        store_region_data = df_store_region[(df_store_region['STORE_TYPE'] == store_type) & 
                                             (df_store_region['REGION'] == region)]
        if not store_region_data.empty:
            first_row = store_region_data.iloc[0]
            store_region_row[f'{region}_ytd_sales_cy'] = first_row['KSA_YTD_SALES_REGION_STORE']
            store_region_row[f'{region}_growth_pct'] = first_row['KSA_GROWTH_PCT_REGION_STORE']
            store_region_row[f'{region}_achievement_pct'] = first_row['KSA_ACHIEVEMENT_REGION_STORE']
        else:
            store_region_row[f'{region}_ytd_sales_cy'] = None
            store_region_row[f'{region}_growth_pct'] = None
            store_region_row[f'{region}_achievement_pct'] = None
    
    # Store-level totals
    store_data = df_store_region[df_store_region['STORE_TYPE'] == store_type]
    if not store_data.empty:
        first_store_row = store_data.iloc[0]
        store_region_row['combined_ytd_sales'] = first_store_row['KSA_YTD_SALES_STORE']
        store_region_row['combined_growth_pct'] = first_store_row['KSA_GROWTH_PCT_STORE']
        store_region_row['combined_achievement_pct'] = first_store_row['KSA_ACHIEVEMENT_STORE']
    else:
        store_region_row['combined_ytd_sales'] = None
        store_region_row['combined_growth_pct'] = None
        store_region_row['combined_achievement_pct'] = None
    
    final_rows.append(store_region_row)

# ============================================================================
# STEP 4: Add Grand Total
# ============================================================================
grand_total_row = {'CONCEPT': 'KSA', 'STORE_TYPE': 'Grand Total'}

for region in regions:
    region_data = df[df['REGION'] == region].iloc[0]
    grand_total_row[f'{region}_ytd_sales_cy'] = region_data['KSA_YTD_SALES_REGION']
    grand_total_row[f'{region}_growth_pct'] = region_data['KSA_GROWTH_PCT_REGION']
    grand_total_row[f'{region}_achievement_pct'] = region_data['KSA_ACHIEVEMENT_REGION']

# Calculate grand total for rightmost column
grand_total_sales = sum([final_rows[-3]['combined_ytd_sales'], 
                         final_rows[-2]['combined_ytd_sales'], 
                         final_rows[-1]['combined_ytd_sales']])
grand_total_row['combined_ytd_sales'] = grand_total_sales
grand_total_row['combined_growth_pct'] = None
grand_total_row['combined_achievement_pct'] = None

final_rows.append(grand_total_row)

# ============================================================================
# STEP 5: Create DataFrame and format with multi-level headers
# ============================================================================
final_df_complete = pd.DataFrame(final_rows)

# Create multi-level column headers
columns_level_0 = ['', '']
columns_level_1 = ['CONCEPT', 'STORE_TYPE']

regions_upper = ['CENTRAL', 'WESTERN', 'EASTERN', 'ECOMM']
metrics = ['YTD Sales', 'Growth %', 'Achievement %']

for region in regions_upper:
    columns_level_0.extend([region] * 3)
    columns_level_1.extend(metrics)

columns_level_0.extend(['CONCEPT + STORE TYPE'] * 3)
columns_level_1.extend(['YTD Sales', 'Growth %', 'Achievement %'])

multi_columns_complete = pd.MultiIndex.from_arrays([columns_level_0, columns_level_1])

# Reorder columns
ordered_columns_complete = ['CONCEPT', 'STORE_TYPE']
for region in ['central', 'western', 'eastern', 'ecomm']:
    ordered_columns_complete.extend([
        f'{region}_ytd_sales_cy',
        f'{region}_growth_pct',
        f'{region}_achievement_pct'
    ])

ordered_columns_complete.extend([
    'combined_ytd_sales',
    'combined_growth_pct',
    'combined_achievement_pct'
])

final_df_formatted = final_df_complete[ordered_columns_complete].copy()
final_df_formatted.columns = multi_columns_complete

print(f"✓ Data formatted successfully")
print(f"Shape: {final_df_formatted.shape}")
print(f"\nLast 10 rows:")
final_df_formatted.tail(10)

In [ ]:
# Export to Excel with merged cells
final_df_indexed = final_df_formatted.set_index([('', 'CONCEPT'), ('', 'STORE_TYPE')])

output_file = 'sales_report_final.xlsx'
final_df_indexed.to_excel(output_file, engine='openpyxl', merge_cells=True)

print(f"✓ Excel file created: {output_file}")
print("\nReport structure:")
print("━" * 60)
print("ROWS:")
print("  • Concept groups with store type breakdowns")
print("  • 'Total' row after each concept")
print("  • KSA section at bottom:")
print("    - ECOMM, MALL, STANDALONE (store+region totals)")
print("    - Grand Total")
print("\nCOLUMNS:")
print("  • Columns A-B: CONCEPT, STORE_TYPE")
print("  • Columns C-N: 4 Regions × 3 metrics")
print("  • Columns O-Q: CONCEPT + STORE TYPE totals")
print("  • Merged cells for region headers and concept names")
print("━" * 60)